In [1]:
!pip install seaborn

In [ ]:
import pickle
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
import random

@st.cache_resource
def train_and_save_models(X_train, y_train):
    # Weak Model
    weak_model = DecisionTreeClassifier(max_depth=1, random_state=42)
    weak_model.fit(X_train, y_train)
    with open("weak_model.pkl", "wb") as f:
        pickle.dump(weak_model, f)

    # Bagging
    bagging_model = BaggingClassifier(
        estimator=DecisionTreeClassifier(max_depth=None), n_estimators=50, random_state=42
    )
    bagging_model.fit(X_train, y_train)
    with open("bagging_model.pkl", "wb") as f:
        pickle.dump(bagging_model, f)

    # AdaBoost
    adb_model = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1), n_estimators=50, random_state=42)
    adb_model.fit(X_train, y_train)
    with open("adb_model.pkl", "wb") as f:
        pickle.dump(adb_model, f)

    # Gradient Boosting
    gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=42)
    gb_model.fit(X_train, y_train)
    with open("gb_model.pkl", "wb") as f:
        pickle.dump(gb_model, f)

    # Stacking
    estimators = [("dt", DecisionTreeClassifier(max_depth=1)), ("svc", SVC(probability=True))]
    stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=5)
    stacking_model.fit(X_train, y_train)
    with open("stacking_model.pkl", "wb") as f:
        pickle.dump(stacking_model, f)

    # Voting
    voting_model = VotingClassifier(
        estimators=[
            ("dt", DecisionTreeClassifier(max_depth=1)),
            ("rf", BaggingClassifier(DecisionTreeClassifier(), n_estimators=10)),
            ("svc", SVC(probability=True))
        ],
        voting="soft"
    )
    voting_model.fit(X_train, y_train)
    with open("voting_model.pkl", "wb") as f:
        pickle.dump(voting_model, f)

    return weak_model, bagging_model, adb_model, gb_model, stacking_model, voting_model

import os

def load_models():
    if all(os.path.exists(fname) for fname in [
        "weak_model.pkl", "bagging_model.pkl", "adb_model.pkl",
        "gb_model.pkl", "stacking_model.pkl", "voting_model.pkl"
    ]):
        with open("weak_model.pkl", "rb") as f:
            weak_model = pickle.load(f)
        with open("bagging_model.pkl", "rb") as f:
            bagging_model = pickle.load(f)
        with open("adb_model.pkl", "rb") as f:
            adb_model = pickle.load(f)
        with open("gb_model.pkl", "rb") as f:
            gb_model = pickle.load(f)
        with open("stacking_model.pkl", "rb") as f:
            stacking_model = pickle.load(f)
        with open("voting_model.pkl", "rb") as f:
            voting_model = pickle.load(f)
        return weak_model, bagging_model, adb_model, gb_model, stacking_model, voting_model
    else:
        return train_and_save_models(X_train, y_train)

st.set_page_config(page_title="Creative Ensemble Demo", layout="wide")
st.title("Creative Ensemble Demo: Weak vs Ensemble Models")

file_path = "cybersecurity_intrusion_data.csv"
df = pd.read_csv(file_path)

target_column = "attack_detected"
X = df.drop(columns=[target_column])
y = df[target_column]

le = LabelEncoder()
y_encoded = le.fit_transform(y)

for col in X.columns:
    if X[col].dtype == object:
        X[col] = LabelEncoder().fit_transform(X[col])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded
)

weak_model, bagging_model, adb_model, gb_model, stacking_model, voting_model = load_models()

# Sidebar for selecting demonstration mode
mode = st.sidebar.selectbox("Demo Mode", ["Model Comparison","Tricky Packets", "Noise Stress Test", "Minority Attack Focus"])


if mode == "Model Comparison":
    st.header("Model Comparison: Accuracy, Precision, Recall, F1-Score, Confusion Matrix")

    models = {
        "Weak Tree": weak_model,
        "Bagging": bagging_model,
        "AdaBoost": adb_model,
        "GradientBoosting": gb_model,
        "Stacking": stacking_model,
        "Voting": voting_model
    }

    # Create a DataFrame to store metrics
    metrics_df = pd.DataFrame(columns=["Model", "Accuracy", "Precision", "Recall", "F1-Score"])

    for name, model in models.items():
        preds = model.predict(X_test)
        metrics_df = metrics_df.append({
            "Model": name,
            "Accuracy": accuracy_score(y_test, preds),
            "Precision": precision_score(y_test, preds, zero_division=0),
            "Recall": recall_score(y_test, preds, zero_division=0),
            "F1-Score": f1_score(y_test, preds, zero_division=0)
        }, ignore_index=True)

    st.subheader("Comparison Table")
    st.dataframe(metrics_df.style.format({
        "Accuracy": "{:.2f}", "Precision": "{:.2f}", "Recall": "{:.2f}", "F1-Score": "{:.2f}"
    }))

    st.subheader("Confusion Matrices")

    # Tabs for confusion matrices
    tabs = st.tabs(list(models.keys()))
    for tab_name, tab in zip(models.keys(), tabs):
        with tab:
            st.write(f"**{tab_name} Confusion Matrix**")
            cm = confusion_matrix(y_test, models[name].predict(X_test))

            fig, ax = plt.subplots(figsize=(1.8, 1.8), dpi=100)

            ax.imshow(cm, cmap="Blues", alpha=0.8)

            for i in range(cm.shape[0]):
                for j in range(cm.shape[1]):
                    ax.text(
                        j, i, cm[i, j],
                        ha="center", va="center",
                        fontsize=7
                    )

            ax.set_xticks([0, 1])
            ax.set_yticks([0, 1])
            ax.set_xticklabels(le.classes_, fontsize=7)
            ax.set_yticklabels(le.classes_, fontsize=7)

            ax.set_xlabel("Predicted", fontsize=7)
            ax.set_ylabel("Actual", fontsize=7)

            plt.tight_layout(pad=0.5)

            st.pyplot(fig, use_container_width=False)

elif mode == "Tricky Packets":
    st.header("Tricky Packets Demo: Weak Model Fails, Ensembles Succeed")
    num_packets = st.slider("Number of tricky packets", 1, 10, 5)
    random_indices = random.sample(range(X_test.shape[0]), num_packets)
    X_tricky = X_test[random_indices]
    y_tricky = y_test[random_indices]

    st.write("We modify some features slightly to confuse weak models...")
    X_modified = X_tricky.copy()
    noise = np.random.normal(0, 0.5, X_modified.shape)
    X_modified += noise

    models = {
        "Weak Tree": weak_model,
        "Bagging": bagging_model,
        "AdaBoost": adb_model,
        "GradientBoosting": gb_model,
        "Stacking": stacking_model,
        "Voting": voting_model
    }

    results = []
    for i in range(num_packets):
        row = X_modified[i].reshape(1, -1)
        true_label = y_tricky[i]
        row_results = {"True": true_label}
        for name, model in models.items():
            pred = model.predict(row)[0]
            row_results[name] = pred
        results.append(row_results)

    df_results = pd.DataFrame(results)
    def color_preds(val, true_val):
        if val == true_val:
            return 'background-color: #b6fcd5'  # green
        else:
            return 'background-color: #fcb6b6'  # red

    st.write("Predictions table (green = correct, red = wrong)")
    st.dataframe(df_results.style.apply(lambda x: [color_preds(v, x['True']) for v in x], axis=1))

elif mode == "Noise Stress Test":
    st.header("Noise Stress Test: Ensembles vs Weak Model")
    noise_levels = [0, 0.1, 0.2, 0.3, 0.5]
    weak_acc = []
    bag_acc = []
    adb_acc = []
    gb_acc = []

    for n in noise_levels:
        X_noisy = X_test + np.random.normal(0, n, X_test.shape)
        weak_acc.append(accuracy_score(y_test, weak_model.predict(X_noisy)))
        bag_acc.append(accuracy_score(y_test, bagging_model.predict(X_noisy)))
        adb_acc.append(accuracy_score(y_test, adb_model.predict(X_noisy)))
        gb_acc.append(accuracy_score(y_test, gb_model.predict(X_noisy)))

    st.line_chart({
        "Noise Level": noise_levels,
        "Weak Model": weak_acc,
        "Bagging": bag_acc,
        "AdaBoost": adb_acc,
        "GradientBoosting": gb_acc
    })

elif mode == "Minority Attack Focus":
    st.header("Minority Attack Detection: Ensembles vs Weak Model")
    attack_class = 1  # assuming 1 = attack
    weak_preds = weak_model.predict(X_test)
    bag_preds = bagging_model.predict(X_test)
    adb_preds = adb_model.predict(X_test)
    gb_preds = gb_model.predict(X_test)

    def attack_detection_rate(y_true, y_pred):
        attack_idx = np.where(y_true == attack_class)[0]
        return np.mean(y_pred[attack_idx] == attack_class)

    st.write(f"Weak Model Attack Detection Rate: {attack_detection_rate(y_test, weak_preds):.2f}")
    st.write(f"Bagging Attack Detection Rate: {attack_detection_rate(y_test, bag_preds):.2f}")
    st.write(f"AdaBoost Attack Detection Rate: {attack_detection_rate(y_test, adb_preds):.2f}")
    st.write(f"GradientBoosting Attack Detection Rate: {attack_detection_rate(y_test, gb_preds):.2f}")
